In [5]:
import pandas as pd
import json
from openai import OpenAI
import random

In [2]:
raw = pd.read_csv("GPTtrainingdata16.csv", header=0)
raw

,describe what this person did during the discussion without your opinion (~50 words),"what kind of role did this person play in the discussion, and how did it help the discussion group? (~50 words)",how can this person improve? (~50 words),"hand made sample output (Leadership, critical thinking, empathy)"
0,Peer evaluation1: Frequently summarized discus...,"Peer evaluation4: Mediator, maintaining focus ...",Peer evaluation7: Further enhance the discussi...,Leadership score 8: Peers highly evaluated his...
1,Peer evaluation1: Consistently introduced new ...,Peer evaluation4: Initiative in directing the ...,Peer evaluation7: Could benefit from allowing ...,Leadership score 6: Peers highly evaluated his...
2,Peer evaluation1: Listened attentively to othe...,"Peer evaluation4: Reconciling role, bridging g...",Peer evaluation7: Improvement could be made by...,Leadership score 4: Peers highly evaluated her...
3,Peer evaluation1: Offered insightful feedback ...,"Peer evaluation4: Thoughtful contributor, enri...",Peer evaluation7: Enhancing her ability to con...,Leadership score 4: Although she actively enga...
4,Peer evaluation1: he effectively summarized ke...,Peer evaluation4: his leadership qualities wer...,Peer evaluation7: he could further enhance the...,Leadership score 8: Peers highly evaluated his...
5,Peer evaluation1: she consistently introduced ...,"Peer evaluation4: she acted as a facilitator, ...",Peer evaluation7: she could benefit from occas...,Leadership score 6: Peers highly evaluated her...
6,Peer evaluation1: she attentively listened to ...,Peer evaluation4: she played the role of a med...,Peer evaluation7: Greater participation in ini...,Leadership score 4: Although peers higly evalu...
7,Peer evaluation1: he consistently challenged a...,Peer evaluation4: he took on the role of a cri...,Peer evaluation7: Balancing his analytical app...,Leadership Score 4: Peers commended his abilit...
8,"Peer evaluation1: He mostly remained silent, b...",Peer evaluation4: He played the role of a rese...,"Peer evaluation7: To enhance participation, he...",Leadership Score 2: While responding well to o...
9,Peer evaluation1: she frequently interjected w...,"Peer evaluation4: She acted as a facilitator, ...",Peer evaluation7: Could engage more by sharing...,Leadership score 4: Although peers higly evalu...


In [3]:
raw.columns = ["peer123", "peer456", "peer789", "sample_output"]
raw


,peer123,peer456,peer789,sample_output
0,Peer evaluation1: Frequently summarized discus...,"Peer evaluation4: Mediator, maintaining focus ...",Peer evaluation7: Further enhance the discussi...,Leadership score 8: Peers highly evaluated his...
1,Peer evaluation1: Consistently introduced new ...,Peer evaluation4: Initiative in directing the ...,Peer evaluation7: Could benefit from allowing ...,Leadership score 6: Peers highly evaluated his...
2,Peer evaluation1: Listened attentively to othe...,"Peer evaluation4: Reconciling role, bridging g...",Peer evaluation7: Improvement could be made by...,Leadership score 4: Peers highly evaluated her...
3,Peer evaluation1: Offered insightful feedback ...,"Peer evaluation4: Thoughtful contributor, enri...",Peer evaluation7: Enhancing her ability to con...,Leadership score 4: Although she actively enga...
4,Peer evaluation1: he effectively summarized ke...,Peer evaluation4: his leadership qualities wer...,Peer evaluation7: he could further enhance the...,Leadership score 8: Peers highly evaluated his...
5,Peer evaluation1: she consistently introduced ...,"Peer evaluation4: she acted as a facilitator, ...",Peer evaluation7: she could benefit from occas...,Leadership score 6: Peers highly evaluated her...
6,Peer evaluation1: she attentively listened to ...,Peer evaluation4: she played the role of a med...,Peer evaluation7: Greater participation in ini...,Leadership score 4: Although peers higly evalu...
7,Peer evaluation1: he consistently challenged a...,Peer evaluation4: he took on the role of a cri...,Peer evaluation7: Balancing his analytical app...,Leadership Score 4: Peers commended his abilit...
8,"Peer evaluation1: He mostly remained silent, b...",Peer evaluation4: He played the role of a rese...,"Peer evaluation7: To enhance participation, he...",Leadership Score 2: While responding well to o...
9,Peer evaluation1: she frequently interjected w...,"Peer evaluation4: She acted as a facilitator, ...",Peer evaluation7: Could engage more by sharing...,Leadership score 4: Although peers higly evalu...


In [4]:
jsonl_format = []
for i in raw.index:
    input_ = raw.iloc[i]["peer123"] +' '+ raw.iloc[i]["peer456"] +' '+ raw.iloc[i]["peer789"]
    output = raw.iloc[i]["sample_output"]
    jsonl_format.append({"messages": [{"role": "system", "content": "You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person."}, {"role": "user", "content": "Following are the peer reviews for this person: " + f"{input_}"}, {"role": "assistant", "content": f"{output}"}]})

jsonl_format
    

[{'messages': [{'role': 'system',
    'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
   {'role': 'user',
    'content': 'Following are the peer reviews for this person: Peer evaluation1: Frequently summarized discussion points and asked clarifying questions, ensuring understanding. Peer evaluation2: Consistently provided structure to the discussion through effective organization of thoughts and ideas. Peer evaluation3: Excelled in maintaining order and ensuring thorough discussion of each point. Peer evaluation4: Mediator, maintaining focus and coherence, especially when the discussion veered off-topic. Peer evaluation5: Instrumental in guiding the flow of conversation and ensuring that key points were addressed and understood. Peer evaluation6: Organizational skills were key in keeping the discussion coherent and productive. Pe

In [9]:
Train_data = random.sample(jsonl_format, 11)
Test_data = [i for i in jsonl_format if i not in Train_data]
print(Train_data)
print("\n")
print(Test_data)

[{'messages': [{'role': 'system', 'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'}, {'role': 'user', 'content': "Following are the peer reviews for this person: Peer evaluation1: Consistently introduced new ideas and asked probing questions, often guiding the conversation towards deeper analysis. Peer evaluation2: Often took the lead in exploring uncharted areas of the topic, adding depth to the conversation. Peer evaluation3: Quick to identify logical inconsistencies and suggest alternative viewpoints. Peer evaluation4: Initiative in directing the discussion, exploring diverse viewpoints and keeping the conversation on track. Peer evaluation5: Proactive approach in steering the discussion helped in uncovering new angles and perspectives. Peer evaluation6: Provided a challenging but constructive voice, pushing the group towards mor

In [10]:
print(len(Train_data), len(Test_data))

11 5


In [14]:
with open('TrainData11S.jsonl', 'w') as f:
    for i in range(len(Train_data)):
        f.writelines(json.dumps(Train_data[i]))
        f.write("\n")

In [15]:
client = OpenAI(api_key="")

client.files.create(
    file=open("TrainData11S.jsonl", "rb"),
    purpose="fine-tune",
)

FileObject(id='file-SZqArdL6AUodBfvN2WsC5JhG', bytes=20617, created_at=1702549273, filename='TrainData11S.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [16]:
client = OpenAI(api_key="")
client.fine_tuning.jobs.create(
  training_file="file-SZqArdL6AUodBfvN2WsC5JhG", 
  model="gpt-3.5-turbo-0613"
)


FineTuningJob(id='ftjob-IyAaJXoxjHZXgJCKbXRmbeTO', created_at=1702549287, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-NEvo5J4DH55S1aLGck9nKzUC', result_files=[], status='validating_files', trained_tokens=None, training_file='file-SZqArdL6AUodBfvN2WsC5JhG', validation_file=None)

In [17]:
Test_data[0]

{'messages': [{'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
  {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: Listened attentively to others, summarizing points effectively and offering supportive comments. Peer evaluation2: Regularly encouraged quieter members to speak and acknowledged all contributions positively. Peer evaluation3: Consistently acknowledged and built upon others' contributions, fostering a collaborative atmosphere. Peer evaluation4: Reconciling role, bridging gaps between differing opinions and maintaining a harmonious discussion environment. Peer evaluation5: Fostered a supportive environment, ensuring that all voices were heard and valued, which enriched the group's collective output. Peer evaluation6: Nurturing role, ensuring that each

In [18]:
instruction = {'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'}

In [19]:
test_input1 = {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: Listened attentively to others, summarizing points effectively and offering supportive comments. Peer evaluation2: Regularly encouraged quieter members to speak and acknowledged all contributions positively. Peer evaluation3: Consistently acknowledged and built upon others' contributions, fostering a collaborative atmosphere. Peer evaluation4: Reconciling role, bridging gaps between differing opinions and maintaining a harmonious discussion environment. Peer evaluation5: Fostered a supportive environment, ensuring that all voices were heard and valued, which enriched the group's collective output. Peer evaluation6: Nurturing role, ensuring that each member felt valued and engaged. Peer evaluation7: Improvement could be made by occasionally challenging ideas to deepen the group's understanding and discussion depth. Peer evaluation8: Greater involvement in the substance of the discussion, rather than just facilitation, could offer more depth to their contributions. Peer evaluation9: Could improve by more actively contributing original ideas and perspectives to the discussion."}


In [21]:
client = OpenAI(api_key="")

response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8VdEkM69",
  messages=[
    instruction,
    test_input1
  ]
)
response

ChatCompletion(id='chatcmpl-8VdztYjl5F6eq8ZmFg9C4gSsfu6it', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Leadership score 0: Many peers highly evaluated leaderhsip, but some peers suggested improvements. Critical Thinking Score 2: Some peers suggeted that critical thinking score can be improved. Empathy score 9: Peers highly evaluated his high empathyc scoring. ', role='assistant', function_call=None, tool_calls=None))], created=1702552609, model='ft:gpt-3.5-turbo-0613:personal::8VdEkM69', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=56, prompt_tokens=245, total_tokens=301))

In [27]:
Test_data[1]

{'messages': [{'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
  {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: she attentively listened to others, offering constructive feedback and summarizing points to clarify understanding. Peer evaluation2: she regularly encouraged others, acknowledging their contributions and helping to maintain a positive atmosphere. Peer evaluation3: she consistently provided thoughtful feedback and was attentive to the emotional tone of the group. Peer evaluation4: she played the role of a mediator, bridging gaps in communication and fostering a collaborative environment. Their approach maintained a positive and productive atmosphere. Peer evaluation5: As a supportive collaborator, she played a vital role in maintaining group morale 

In [28]:
test_input2 = {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: she attentively listened to others, offering constructive feedback and summarizing points to clarify understanding. Peer evaluation2: she regularly encouraged others, acknowledging their contributions and helping to maintain a positive atmosphere. Peer evaluation3: she consistently provided thoughtful feedback and was attentive to the emotional tone of the group. Peer evaluation4: she played the role of a mediator, bridging gaps in communication and fostering a collaborative environment. Their approach maintained a positive and productive atmosphere. Peer evaluation5: As a supportive collaborator, she played a vital role in maintaining group morale and ensuring a harmonious discussion environment. Peer evaluation6: her empathetic presence brought a sense of balance and consideration for others' feelings, which was crucial in maintaining a positive group dynamic. Peer evaluation7: Greater participation in initiating discussion points could enhance her contribution to the group dynamics. Peer evaluation8: she could contribute more of their own ideas to the discussion, adding depth to their supportive role. Peer evaluation9: Being more assertive in sharing their own perspectives could further enhance her role in the group."}

In [29]:
client = OpenAI(api_key="")

response2 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8VdEkM69",
  messages=[
    instruction,
    test_input2
  ]
)
response2

ChatCompletion(id='chatcmpl-8Ve6Tp2MqUEGJseuircVcDyT3z0cs', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Leadership score 2: While she played an important role in supporting and maintaining the group's positive dynamic, her participation in initiating discussion points was limited. Critical thinking score 3: Peers suggested that she could share more of her own ideas. Empathy score 10: Peers highly evaluated her ability to attentively listen and be considerate of others' feelings, which played a crucial role in maintaining the positive group dynamic.", role='assistant', function_call=None, tool_calls=None))], created=1702553017, model='ft:gpt-3.5-turbo-0613:personal::8VdEkM69', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=87, prompt_tokens=268, total_tokens=355))

In [30]:
Test_data[2]

{'messages': [{'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
  {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: He mostly remained silent, but offered well-thought out answers to the questions when the discussion came to him. Peer evaulation2: He demonstrated keen listening skills and provided insightful responses when prompted. His thoughtful contributions showcased depth of understanding. Peer evaluation3: He conveyed a strong grasp of the topic. His well-considered responses indicated a reflective approach, contributing positively to the discussion's overall quality. Peer evaluation4: He played the role of a reserved, yet thoughtful contributor. His well-articulated answers enriched the discussion, offering valuable insights that deepened our understanding

In [31]:
test_input3 = {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: He mostly remained silent, but offered well-thought out answers to the questions when the discussion came to him. Peer evaulation2: He demonstrated keen listening skills and provided insightful responses when prompted. His thoughtful contributions showcased depth of understanding. Peer evaluation3: He conveyed a strong grasp of the topic. His well-considered responses indicated a reflective approach, contributing positively to the discussion's overall quality. Peer evaluation4: He played the role of a reserved, yet thoughtful contributor. His well-articulated answers enriched the discussion, offering valuable insights that deepened our understanding of the topic. Peer evaluation5: Operating as a reflective participant, he contributed a calm and insightful presence. His measured responses fostered a more focused and contemplative atmosphere. Peer evaluation6: He took the role of a contemplative listener, His approach encouraged a more measured and considerate tone in the discussion, enhancing the overall group interaction. Peer evaluation7: To enhance participation, he could proactively share insights rather than relying on being prompted. Peer evaluation8: Initiating discussion points would strengthen his role as a reflective participant, fostering a more dynamic and interactive group environment. Peer evaluation9: While showcasing a strong grasp of the topic, he could share more original opinions rather than simply expanding on the comments of other people."}


In [32]:
client = OpenAI(api_key="")

response3 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8VdEkM69",
  messages=[
    instruction,
    test_input3
  ]
)
response3

ChatCompletion(id='chatcmpl-8VeqTwftm67bIEXWI9sAbpdEzPgJB', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Leadership score 2:  Peers highly evaluated his initiavted discussion and dynamic approach, however, some peers suggested he should proactivley share opinions without being prompted. Critical thinking score 8: Peers highly evalutaed his ability to expand opinions, while empathys score 6: Peers, while recognizing his high ability to listen to other people opinion, also pointed out he didn't actively shared  his own opinion. ", role='assistant', function_call=None, tool_calls=None))], created=1702555869, model='ft:gpt-3.5-turbo-0613:personal::8VdEkM69', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=90, prompt_tokens=318, total_tokens=408))

In [33]:
Test_data[3]

{'messages': [{'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
  {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: she contributed evenly, balancing speaking and listening. She often acknowledged others' points. Peer evaluation2: she spoke up regularly, offering constructive feedback and support to others' ideas. Peer evaluation3: she was active but not overbearing, often providing positive reinforcement to ideas. Peer evaluation4: she was a mediator, maintaining a balanced and inclusive atmosphere. Peer evaluation5: Her role was that of a supporter, reinforcing and building upon others' contributions. Peer evaluation6: She acted as an encourager, boosting the group's morale and fostering collaboration. Peer evaluation7: Could be more assertive in putting forwar

In [34]:
test_input4 = {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: she contributed evenly, balancing speaking and listening. She often acknowledged others' points. Peer evaluation2: she spoke up regularly, offering constructive feedback and support to others' ideas. Peer evaluation3: she was active but not overbearing, often providing positive reinforcement to ideas. Peer evaluation4: she was a mediator, maintaining a balanced and inclusive atmosphere. Peer evaluation5: Her role was that of a supporter, reinforcing and building upon others' contributions. Peer evaluation6: She acted as an encourager, boosting the group's morale and fostering collaboration. Peer evaluation7: Could be more assertive in putting forward her ideas. Peer evaluation8: Might try taking the lead in discussions more often. Peer evaluation9: Could occasionally challenge ideas to provoke deeper thinking."}


In [35]:
client = OpenAI(api_key="")

response4 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8VdEkM69",
  messages=[
    instruction,
    test_input4
  ]
)
response4

ChatCompletion(id='chatcmpl-8Ves4nPvwtzKs1NliXjREUNgZWmZC', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Leadership score 2: Although some peers highly evaluated her contributions, some peers pointed out that she could take more initiative in putting forward her ideas. Critical thinking score 5: She did not actively challenge ideas, but she also did not hinder critical discusion. Empathy score 8: Her empathy skills were high, as she often reinforced and acknowledged others' contributions.", role='assistant', function_call=None, tool_calls=None))], created=1702555968, model='ft:gpt-3.5-turbo-0613:personal::8VdEkM69', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=75, prompt_tokens=209, total_tokens=284))

In [36]:
Test_data[4]

{'messages': [{'role': 'system',
   'content': 'You are group of human resource management professionals. You quantify leadership, critical thinking skills, and empathy of a person with reasons according to the peer reviews of that person.'},
  {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: he dominated the conversation, sharing many ideas and opinions, sometimes interrupting others. Peer evaluation2: he was very vocal, offering numerous suggestions and often steering the conversation. Peer evaluation3: he spoke the most, presenting many ideas, but sometimes at the expense of others' participation. Peer evaluation4: As the main speaker, he drove much of the conversation, sparking debate and discussion. Peer evaluation5: Acted as a leader, guiding the direction of the discussion. Peer evaluation6: He was the primary initiator of topics, which kept the conversation lively. Peer evaluation7: Needs to work on listening more and allowing ot

In [38]:
test_input5 = {'role': 'user',
   'content': "Following are the peer reviews for this person: Peer evaluation1: he dominated the conversation, sharing many ideas and opinions, sometimes interrupting others. Peer evaluation2: he was very vocal, offering numerous suggestions and often steering the conversation. Peer evaluation3: he spoke the most, presenting many ideas, but sometimes at the expense of others' participation. Peer evaluation4: As the main speaker, he drove much of the conversation, sparking debate and discussion. Peer evaluation5: Acted as a leader, guiding the direction of the discussion. Peer evaluation6: He was the primary initiator of topics, which kept the conversation lively. Peer evaluation7: Needs to work on listening more and allowing others to contribute equally. Peer evaluation8: Could benefit from being more inclusive and encouraging others to speak. Peer evaluation9: Should focus on balancing his contributions with active listening."}


In [39]:
client = OpenAI(api_key="")

response5 = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8VdEkM69",
  messages=[
    instruction,
    test_input5
  ]
)
response5

ChatCompletion(id='chatcmpl-8VetWKKluNPfOaR6Dz2vEJo42qOg7', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Leadership score 10: Peers highly evaluated his motivation to lead the discussion. Critical thinking score 5: Some peers evaluated his ability to present ideas, but peers also pointed out he should listen more to others. Empahty score 2: Peers did not evaluate his ability to encourage others to speak.', role='assistant', function_call=None, tool_calls=None))], created=1702556058, model='ft:gpt-3.5-turbo-0613:personal::8VdEkM69', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=217, total_tokens=282))